In [2]:
%load_ext watermark
%watermark

2019-05-23T10:12:50+01:00

CPython 3.6.8
IPython 7.5.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-50-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


## Usando Pandas de forma iterativa para procesar datasets medianos (Medium Data)

Podemos usar pandas de forma iterativa para procesar un dataset linea por linea. Ésto sólo se puede hacer para aquellos formatos de datos donde se pueden leer bloques de forma independiente (por ejemplo csv).

Generalmente, para poder procesar datasets de forma iterativa tenemos que pensar en la forma de poder obtener información mediante la partición del dataset en trozos más pequeños (más manejables).

Vamos a trabajar con el dataset [NYC taxi trips dataset 2014](https://data.cityofnewyork.us/Transportation/2014-Yellow-Taxi-Trip-Data/gkne-dk5s) que incluye informacion sobre TODOS los trayectos en taxi en la ciudad de nueva york en 2014. 

Vamos a trabajar con un dataset reducido (obtenido de [aquí](https://www.kaggle.com/kentonnlp/2014-new-york-city-taxi-trips) para no tener que gastar mucho espacio.

In [3]:
ls -alh ../data

total 13G
drwxrwxrwx 1 root root 4.0K May 22 11:37 ./
drwxrwxrwx 1 root root 4.0K May 21 16:38 ../
drwxrwxrwx 1 root root  64K May 22 11:47 2017_Yellow_Taxi_Trip_Data.2.parquet/
-rwxrwxrwx 1 root root 9.9G Feb  1 13:49 2017_Yellow_Taxi_Trip_Data.csv*
drwxrwxrwx 1 root root  32K Feb 19 10:12 2017_Yellow_Taxi_Trip_Data.parquet/
drwxrwxrwx 1 root root 8.0K Mar  4 09:43 coste_npasajeros_2017.csv/
-rwxrwxrwx 1 root root 2.4G Jul 20  2017 nyc_taxi_data_2014.csv*
drwxrwxrwx 1 root root 8.0K May 21 19:49 nyc_taxi_data_2014.parquet/
-rwxrwxrwx 1 root root 770K May 14  2018 posts.jl*


Vemos que el archivo `nyc_taxi_data_2014.csv` ocupa 2.4 Gigabytes, este dataset es moderadamente grande, esto significa que si hacemos:
    
`taxi_2014 = pd.read_csv("../data/nyc_taxi_data_2014.csv")`
Es posible que nuestro ordenador explote (nos daria un `OutOfMemoryError`) al intentar almacenar en la memoria (RAM) de nuestro ordenador todo el dataset.

La función `read_csv` tiene un argumento `nrows` que carga solo las n primeras lineas del dataset, esto nos permite inspeccionar un dataset grande sin quedarnos sin memoria.

In [4]:
import pandas as pd

path_dataset = "../data/nyc_taxi_data_2014.csv"
taxi_2014 = pd.read_csv(path_dataset, nrows=1000)

In [5]:
taxi_2014.shape

(1000, 18)

In [6]:
taxi_2014.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0.0,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0.0,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0.0,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0.0,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0.0,8.75


Cómo podemos iterar la lectura de un archivo csv con pandas? 

Aparte del argumento `nrows` que lee n filas, la función `read_csv` también tiene el argumento `skiprows`, que empieza a leer el archivo saltandose un número determinado de lineas. Con éstos dos argumentos podemos ir leyendo el dataset en bloques (de tamaño nrows) de forma iterativa.

In [7]:
pd.read_csv(path_dataset, nrows=1000, skiprows=1000).head()

,CMT,2014-01-09 17:28:32,2014-01-09 17:41:00,1,1.1000000000000001,-73.995667999999995,40.724729000000004,1.1,N,-73.981593000000004,40.721252999999997,CRD,7.5,1.2,0.5,0.75,0,9.75
0,CMT,2014-01-09 17:27:32,2014-01-09 17:35:34,1,2.0,-74.008188,40.723377,1,N,-73.994775,40.750303,CRD,8.5,1.0,0.5,2.80,0.0,12.80
1,CMT,2014-01-09 17:26:46,2014-01-09 17:43:06,1,1.8,-73.993096,40.727905,1,N,-73.992715,40.748542,CRD,12.0,1.0,0.5,2.70,0.0,16.20
2,CMT,2014-01-09 17:34:46,2014-01-09 17:40:23,1,0.9,-73.962903,40.774955,1,N,-73.950876,40.775706,CRD,5.5,1.0,0.5,1.00,0.0,8.00
3,CMT,2014-01-09 17:36:26,2014-01-09 17:43:00,1,0.9,-73.991148,40.730836,1,N,-73.999496,40.739415,CRD,6.5,1.0,0.5,1.60,0.0,9.60
4,CMT,2014-01-09 17:34:13,2014-01-09 17:45:27,1,1.6,-73.981936,40.778354,1,N,-73.967661,40.771657,CRD,9.5,1.0,0.5,2.75,0.0,13.75


Vemos que al empezar a leer el csv por la linea 10001, pandas no tiene acceso a la cabecera (header) con los nombres de las columnas. Tenemos que indicarselo a pandas (con el argumento `names` indicamos las columnas y con el argumento `header=None` le decimos a read_csv que la primera linea que lea no son las columnas sino una fila normal.

In [8]:
columnas = pd.read_csv(path_dataset, nrows=10).columns
columnas

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount',
       'tolls_amount', 'total_amount'],
      dtype='object')

Ahora podemos empezar a leer un bloque de 1000 filas empezando por la fila 10001

hacemos 1001 en vez de 1000 dado que la ultima fila del primer bloque coincide con la primera fila del segundo bloque. Para evitar ésto, tenemos que saltar el tamaño de bloque (1000) **más una fila más**

In [10]:
pd.read_csv(path_dataset, nrows=1000,skiprows=1000+1, header=None, names=columnas).head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 17:27:32,2014-01-09 17:35:34,1,2.0,-74.008188,40.723377,1,N,-73.994775,40.750303,CRD,8.5,1.0,0.5,2.80,0.0,12.80
1,CMT,2014-01-09 17:26:46,2014-01-09 17:43:06,1,1.8,-73.993096,40.727905,1,N,-73.992715,40.748542,CRD,12.0,1.0,0.5,2.70,0.0,16.20
2,CMT,2014-01-09 17:34:46,2014-01-09 17:40:23,1,0.9,-73.962903,40.774955,1,N,-73.950876,40.775706,CRD,5.5,1.0,0.5,1.00,0.0,8.00
3,CMT,2014-01-09 17:36:26,2014-01-09 17:43:00,1,0.9,-73.991148,40.730836,1,N,-73.999496,40.739415,CRD,6.5,1.0,0.5,1.60,0.0,9.60
4,CMT,2014-01-09 17:34:13,2014-01-09 17:45:27,1,1.6,-73.981936,40.778354,1,N,-73.967661,40.771657,CRD,9.5,1.0,0.5,2.75,0.0,13.75


Ya tenemos una manera de procesar el dataframe de forma iterativa. Lo único que tenemos que montar es un bucle que itere sobre el dataset y cuando acabe de leerlo (que no lo sabemos de antemano dado que no sabemos el número de filas) que pare.

<hr>

**¿Cuantas filas tiene el archivo?**

Para calcular el numero total de filas del archivosolo tenemos que crear una operacion que cuente el numero de filas en un bloque de datos.

para facilitarnos la vida, vamos a hacer una funcion que nos ayude a leer bloques del dataset

In [39]:
def leer_bloque(filas_bloque, skiprows):
    return pd.read_csv(path_dataset, 
                          nrows=filas_bloque, 
                          header=None, 
                          skiprows=skiprows+1, # +1 por el header
                          names=columnas,
                      )

data_block = leer_bloque(filas_bloque=10, skiprows=0)
data_block.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0,8.75


Solo falta el crear la operacion que queremos hacer para cada bloque

In [14]:
def n_filas_bloque(bloque):
    return bloque.shape[0]

Ahora creamos un bucle que lea bloques de datos del archivo hasta que no haya mas, y para cada uno ejecute la funcion que calcula el numero de filas

In [15]:
%%time
filas_bloque = 1000000
n_bloque = 0
n_total_filas = 0

while True:
    bloque = leer_bloque(filas_bloque=filas_bloque, skiprows=filas_bloque * n_bloque)
    if bloque.empty:
        break
    else:
        n_total_filas+= n_filas_bloque(bloque)
    print(n_bloque, n_total_filas)
    n_bloque += 1

0 1000000
1 2000000
2 3000000
3 4000000
4 5000000
5 6000000
6 7000000
7 8000000
8 9000000
9 10000000
10 11000000
11 12000000
12 13000000
13 14000000


/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


14 14999999
CPU times: user 2min 30s, sys: 7.12 s, total: 2min 37s
Wall time: 2min 41s


In [16]:
n_total_filas

14999999

Vemos que hay un total de 15 millones de filas

<hr>

**¿Cuantos trayectos hay por tipo de pago??**


Ahora podemos implementar una operacion mas compleja, en la que calculamos el numero total de trayectos por tipo de pago de viaje.


La operacion es similar a la que hemos hecho con el calculo de numero de filas. Ahora simplemente tenemos que mantener un diccionario con el numero de filas por tipo de pago y actualizarlo en cada bloque.

Usamos un `defaultdict` por que no sabemos todos los valores distintos de tipo de pago

In [34]:
from collections import defaultdict

n_trayectos_tipo_pago = defaultdict(lambda: 0)
n_trayectos_tipo_pago

defaultdict(<function __main__.<lambda>()>, {})

In [35]:
def calculo_trayectos_tipo_pago(bloque):
    bloque_trayectos_tipo_pago = bloque.payment_type.value_counts().to_dict()
    for tipo_pago, n_trayectos in bloque_trayectos_tipo_pago.items():
        n_trayectos_tipo_pago[tipo_pago] += n_trayectos

In [36]:
filas_bloque = 1000000 #reduce el numero en caso de fallo por falta de RAM
n_bloque = 0

while 1:
    bloque = leer_bloque(filas_bloque=filas_bloque, skiprows=filas_bloque * n_bloque)
    if bloque.empty:
        break
    else:
        calculo_trayectos_tipo_pago(bloque)
    n_bloque += 1
    print(n_bloque * filas_bloque)

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000


/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


15000000


In [37]:
n_trayectos_tipo_pago

defaultdict(<function __main__.<lambda>()>,
            {'CSH': 6215019,
             'CRD': 8667410,
             'NOC': 33737,
             'DIS': 9850,
             'UNK': 73983})

<hr>

**¿Cual es el valor medio de las propinas por tipo de pago?**

Esta operacion es más compleja, ya que la media depende de los datos de todo el dataset, no solo de cada bloque. Para calcular la media tenemos que calcular la suma total de propinas para todos los trayectos por tipo de pago y luego dividir por el numero de trayectos por tipo de pago(que acabamos de calcular)

In [40]:
%%time

def calculo_propina_tipo_pago(bloque):
    tipo_pago_bloque = bloque.groupby("payment_type")["tip_amount"].sum().to_dict()
    for tipo_pago, suma_propina in tipo_pago_bloque.items():
        suma_propinas_tipo_pago[tipo_pago] += suma_propina
        
        
filas_bloque = 1000000 # iteramos sobre un 1millon a la vez, esto depende de vuestro ordenador
n_bloque = 0
suma_propinas_tipo_pago = defaultdict(lambda: 0.0) # las propinas tienen decimales

while 1:
    bloque = leer_bloque(filas_bloque=filas_bloque, skiprows=filas_bloque * n_bloque)
    if bloque.empty:
        break
    else:
        n_total_filas = calculo_propina_tipo_pago(bloque)
    n_bloque += 1
    print(n_bloque * filas_bloque)

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
CPU times: user 2min 53s, sys: 9.65 s, total: 3min 3s
Wall time: 3min 3s


In [41]:
suma_propinas_tipo_pago

defaultdict(<function __main__.<lambda>()>,
            {'CRD': 21674215.160005085,
             'CSH': 2040.65,
             'DIS': 101.60000000000001,
             'NOC': 162.5,
             'UNK': 162086.78999999963})

Ahora simplemente dividimos las sumas por el numero de trayectos para obtener la propina media por tipo de pago

In [42]:
n_trayectos_tipo_pago

defaultdict(<function __main__.<lambda>()>,
            {'CSH': 6215019,
             'CRD': 8667410,
             'NOC': 33737,
             'DIS': 9850,
             'UNK': 73983})

In [43]:
propina_media = {}
for tipo_pago, suma_propinas in suma_propinas_tipo_pago.items():
    propina_media[tipo_pago] = suma_propinas / n_trayectos_tipo_pago[tipo_pago]

y ya está!, ya hemos calculado la propina media por tipo de pago sin tener que cargar todo el dataset

In [44]:
propina_media

{'CRD': 2.500656500616111,
 'CSH': 0.00032834171544769214,
 'DIS': 0.010314720812182742,
 'NOC': 0.004816670124788807,
 'UNK': 2.1908653339280595}

<hr>

**Como podemos guardar los trayectos en metalico**

Una forma muy sencilla de guardar los trayectos es procesar cada bloque, filtrar lo que nos interesa y guardarlo en otro archivo

In [58]:
def escribir_bloque(bloque, output):
    bloque.to_csv(output, mode="a", index=False, header=False)

def guardar_en_metalico(bloque, output):
    bloque_en_metalico = bloque[bloque.payment_type=="CSH"]
    if not bloque_en_metalico.empty:
        escribir_bloque(bloque_en_metalico, output)
    return bloque_en_metalico.shape[0]

Creamos ahora un archivo de output y escribimos una sola fila con los nombres de las columnas

In [73]:
output = "../data/taxi_2014_en_metalico.csv"
with open(output, "w") as fname:
    fname.write(",".join(columnas))
    fname.write("\n")

In [74]:
%%time

filas_bloque = 1000000 # iteramos sobre un 1millon a la vez, esto depende de vuestro ordenador
n_bloque = 0
    
    
while 1:
    bloque = leer_bloque(filas_bloque=filas_bloque, skiprows=filas_bloque * n_bloque)
    if bloque.empty:
        break
    else:
        filas_escritas = guardar_en_metalico(bloque, output)
    n_bloque += 1
    print(n_bloque * filas_bloque, filas_escritas)

1000000 545819
2000000 361613
3000000 405678
4000000 388565
5000000 480235
6000000 581422
7000000 19362
8000000 561960
9000000 404067
10000000 412082
11000000 406424
12000000 360718
13000000 288054
14000000 697524
15000000 301496
CPU times: user 4min 6s, sys: 10.7 s, total: 4min 17s
Wall time: 4min 30s


In [76]:
pd.read_csv(output, nrows=100).head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2014-01-20 00:25:00,2014-01-20 00:59:00,6,14.09,-73.991490,40.754865,1,NaN,-73.851190,40.836462,CSH,42.5,0.5,0.5,0.0,0.0,43.5
1,VTS,2014-01-20 00:59:00,2014-01-20 01:04:00,2,1.46,-73.989547,40.762525,1,NaN,-74.002425,40.744772,CSH,6.5,0.5,0.5,0.0,0.0,7.5
2,VTS,2014-01-20 01:01:00,2014-01-20 01:04:00,2,0.98,-73.989740,40.760560,1,NaN,-73.991880,40.749930,CSH,5.0,0.5,0.5,0.0,0.0,6.0
3,VTS,2014-01-20 00:55:00,2014-01-20 01:10:00,6,2.87,-73.958850,40.709482,1,NaN,-73.916173,40.689455,CSH,13.0,0.5,0.5,0.0,0.0,14.0
4,VTS,2014-01-20 01:03:00,2014-01-20 01:10:00,1,2.04,-73.980668,40.730045,1,NaN,-73.992217,40.749160,CSH,8.5,0.5,0.5,0.0,0.0,9.5


In [77]:
pd.read_csv(output).payment_type.value_counts()

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


CSH    6215019
Name: payment_type, dtype: int64